### RGPD exclusion process
- Take the 2024 beneficiaries data into a dataframe
- Exclude from this 2024 beneficiaries dataframe the people that are inside
  - The list of codes to exclude
  - The list of emails to exclude
- Create a CSV file with the excluded rgpd lists

In [ ]:
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()

pathfile_beneficiaries_2024 = os.environ['BENEF_2024_PATHFILE']
pathfile_duplicates_to_exclude_benef_2024= os.environ['BENEF_2024_DUPLICATES_TO_EXCLUDE_PATHFILE']
pathfile_rgpd_emails_to_exclude =  os.environ['RGPD_EMAILS_TO_EXCLUDE_PATHFILE']
pathfile_rgpd_codes_to_exclude =  os.environ['RGPD_CODES_TO_EXCLUDE_PATHFILE']

output_pathfile_filtered_beneficiaries = os.environ['BENEF_2025_RGPD_DUPLICATES_FILTERED_OUTPUT_PATHFILE']

df_2024 = pd.read_csv(pathfile_beneficiaries_2024)
df_rgpd_emails_to_exclude = pd.read_csv(pathfile_rgpd_emails_to_exclude)
df_rgpd_codes_to_exclude = pd.read_csv(pathfile_rgpd_codes_to_exclude)
df_duplicates = pd.read_csv(pathfile_duplicates_to_exclude_benef_2024)

In [ ]:
import json

# Extract emails from json
df_json_allocataire = pd.json_normalize(df_2024['allocataire'].apply(json.loads)).add_prefix('allocataire-')
df_2024_unwrapped = pd.merge(df_2024, df_json_allocataire['allocataire-courriel'], left_index=True, right_index=True)

In [ ]:
emails_to_exclude_mask = ~df_2024_unwrapped['allocataire-courriel'].isin(df_rgpd_emails_to_exclude['email'])
codes_to_exclude_mask = ~df_2024_unwrapped['id_psp'].isin(df_rgpd_codes_to_exclude['id_psp'])
duplicate_mask = ~df_2024_unwrapped['id'].isin(df_duplicates['id'])

In [ ]:
df_with_excluded_data_2024 = df_2024_unwrapped[emails_to_exclude_mask & codes_to_exclude_mask & duplicate_mask]

In [ ]:
assert(len(df_2024_unwrapped) > len(df_with_excluded_data_2024))

In [ ]:
df_with_excluded_data_2024.to_csv(output_pathfile_filtered_beneficiaries, index=False)